In [ ]:
import numpy as np
import lmfit

from pathlib import Path

In [ ]:
# Configuration
potentials = ["an_cai", "da1p"]
states = ["gs", "0.32", "1.78", "2.65", "3.40n", "3.40p"]
energies = ["win", "mid", "mm"]
theory_dir = Path("C:\\Users\\zachs\\Desktop\\data4plots\\theory_calculations")
xs_dir = Path("C:\\Users\\zachs\\Desktop\\data4plots\\xs_gate_err\\xs_with_err")
write_path = Path("C:\\Users\\zachs\\Desktop")

In [ ]:
results = np.empty((len(potentials), len(states), len(energies)))
for dp, potential in enumerate(potentials):
    for ds, state in enumerate(states):
        for de, energy in enumerate(energies):
            theory = np.load(theory_dir / potential / state / f"{energy}.npz")

            if state == states[-1] or state == states[-2]:
                xs = np.genfromtxt(xs_dir / f"dp_{states[-1][:-1]}_xs_ep.csv", delimiter=',', skip_header=1)
            else:
                xs = np.genfromtxt(xs_dir / f"dp_{state}_xs_ep.csv", delimiter=',', skip_header=1)

            theory_fit = lambda x, A: A * np.interp(x, theory["angle"], theory["cross"])
            params = lmfit.Parameters()
            params.add("A", value=1, min=0)
            lineshape = lmfit.Model(theory_fit)

            fit_weights = np.sqrt(xs[:, 3])
            fit_weights = np.divide(1, fit_weights, out=np.zeros_like(fit_weights), where=fit_weights!=0.0)
            fit = lineshape.fit(params=params, x=xs[:, 0], data=xs[:, 2], weights=fit_weights)

            results[dp, ds, de] = fit.params["A"].value

In [ ]:
# Choose potential to look at
for set in results[0]:
    print(f"{set[1]:.2f}, {(set[0]-set[1]):.2f}, {(set[2]-set[1]):.2f}")

In [ ]:
# Calculate averages
averages = np.average(results, axis=0)
for set in averages:
    print(f"{set[1]:.2f}, {(set[0]-set[1]):.2f}, {(set[2]-set[1]):.2f}")